[<img src="../header.svg">](../index.ipynb)

---

# Simulation of Drug Transport across a Virtual Skin Membrane

### Initialize UG4 (for 2D and standard algebra)

In [9]:
InitUG(2, AlgebraType("CPU", 1));
ug_load_script("ug_util.lua")
ug_load_script("util/refinement_util.lua")

### Create Domain

In [48]:
requiredSubsets = {"FREE", "BARRIER", "NUCLEUS", "MEMBRANE"}
gridName = "simple-cell.ugx"
numRefs = 2

In [37]:
dom = util.CreateDomain(gridName, numRefs, requiredSubsets)

Loading Domain simple-cell.ugx ... done.
Performing integrity check on domain ... done.
Refining(3): 1 2 3 done.


### Create Approximation space

In [38]:
approxSpaceDesc = { fct = "c", type = "Lagrange", order = 1 }

In [39]:
approxSpace = ApproximationSpace(dom)
approxSpace:add_fct(approxSpaceDesc.fct, approxSpaceDesc.type, approxSpaceDesc.order)
approxSpace:init_levels()
approxSpace:init_top_surface()
print("Approximation space:")
approxSpace:print_statistic()

Approximation space:
| ----------------------------------------------------------------------------------------- |
|  Number of DoFs (All Procs)                                                               |
|  Algebra: Block 1 (divide by 1 for #Index)                                                |
|                                                                                           |
|    GridLevel   |       Domain |      0: FREE |   1: BARRIER |   2: NUCLEUS |  3: MEMBRANE |
| ----------------------------------------------------------------------------------------- |
| (lev,    0)    |          216 |            9 |          153 |            6 |           48 |
| (lev,    1)    |          810 |          252 |          450 |           12 |           96 |
| (lev,    2)    |         3132 |         1431 |         1485 |           24 |          192 |
| (lev,    3)    |        12312 |         6561 |         5319 |           48 |          384 |
| (lev,    0, g) |          216 |      

## Create a convection-diffusion-equation
$$\frac{\partial Ku}{\partial t} + \nabla \cdot [-DK \nabla u] = 0$$
Model parameter

In [40]:
K={
    ["FREE"] = 1.0, ["BARRIER"] = 1.0,
}

D={
     ["FREE"] = 1, ["BARRIER"] = 0.001, 
}


In [41]:
elemDisc ={}

elemDisc["BARRIER"] = ConvectionDiffusion("c", "BARRIER", "fv1")
elemDisc["BARRIER"]:set_diffusion(K["BARRIER"]*D["BARRIER"])
elemDisc["BARRIER"]:set_mass_scale(K["BARRIER"])

elemDisc["FREE"] = ConvectionDiffusion("c", "FREE", "fv1")
elemDisc["FREE"]:set_diffusion(K["FREE"]*D["FREE"])
elemDisc["FREE"]:set_mass_scale(K["FREE"])

In [42]:
dirichletBnd = DirichletBoundary()
dirichletBnd:add(1.0, "c", "NUCLEUS")
dirichletBnd:add(0.0, "c", "MEMBRANE")

In [43]:
domainDisc = DomainDiscretization(approxSpace)
domainDisc:add(elemDisc["FREE"])
domainDisc:add(elemDisc["BARRIER"])
domainDisc:add(dirichletBnd)

## Create  solver

In [44]:
solver=LU()

## Solve steady-state problem

In [45]:
A = AssembledLinearOperator(domainDisc)
x = GridFunction(approxSpace)
b = GridFunction(approxSpace)
x:set(0.0)


domainDisc:assemble_linear(A, b)
domainDisc:adjust_solution(x)

solver:init(A, x)
solver:apply(x, b)


ILUT: please use 'set_ordering_algorithm(..)' in the future

Using ILUT(0) on 12312 x 12312 matrix...
[-------------------------25%----------------------50%----------------------75%----------------------] took 12.709 s.


In [46]:
WriteGridFunctionToVTK(x, "SimpleCellRef"..numRefs..".vtu")

In [47]:
flux = IntegrateNormalGradientOnManifold(x, "c", "MEMBRANE", "FREE")
print(flux)

0.083582095669179
